# N8N Workflow Generator - Kaggle Training Notebook

This notebook is optimized for Kaggle. It will:
- Check GPU availability
- Install required packages
- Load your uploaded dataset from `/kaggle/input/`
- Train the model with a VS Code-friendly progress bar
- Save checkpoints and final model to `/kaggle/working/`

**How to use:**
1. Upload your dataset as a Kaggle dataset and add it via 'Add Data'
2. Run all cells in order
3. Monitor progress in the output and `/kaggle/working/training_progress.log`
4. Download the final model from the Output tab

In [1]:
# Check GPU availability
!nvidia-smi

Thu Oct 16 13:23:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers==4.44.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [3]:
# Install required packages
!pip install -q datasets peft accelerate bitsandbytes scipy trl torch tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# List available datasets in /kaggle/input/
import os
print('Datasets in /kaggle/input/:')
for item in os.listdir('/kaggle/input'):
    print(' -', item)

Datasets in /kaggle/input/:
 - testingdata


In [5]:
# Load your dataset (update path if needed)
import json
from datasets import Dataset
dataset_path = '/kaggle/input/testingdata/dataset.jsonl'  # Change if needed
formatted_data = []
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            item = json.loads(line.strip())
            workflow_str = json.dumps(item['workflow']) if isinstance(item['workflow'], dict) else item['workflow']
            formatted_data.append({
                'text': f'''<|system|>
You are an n8n workflow generator. Convert natural language descriptions into valid n8n workflow JSON.
<|user|>
{item['prompt']}
<|assistant|>
{workflow_str}'''
            })
train_dataset = Dataset.from_list(formatted_data)
print(f'Loaded {len(train_dataset)} examples')

Loaded 1000 examples


In [6]:
# Load model and tokenizer (after restart)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type='nf4', 
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    device_map='auto'
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
print('✅ Model loaded successfully!')

2025-10-16 13:25:51.832070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760621152.208809      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760621152.329096      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

✅ Model loaded successfully!


In [7]:
# Configure LoRA adapters
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


In [8]:
# Training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/n8n-workflow-generator',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    save_strategy='steps',
    save_steps=25,
    save_total_limit=3,
    logging_steps=5,
    warmup_steps=100,
    optim='paged_adamw_8bit',
    max_grad_norm=0.3,
    lr_scheduler_type='cosine',
    report_to='none',
    logging_first_step=True,
    disable_tqdm=False,
    gradient_checkpointing=True,
)
print('Training arguments configured!')

Training arguments configured!


In [9]:
# Training cell with VS Code-friendly progress bar and log file
from transformers import Trainer, DataCollatorForLanguageModeling, TrainerCallback
import time
import os

class TextProgressCallback(TrainerCallback):
    def __init__(self, total_steps, epochs, log_path):
        self.total_steps = total_steps
        self.epochs = epochs
        self.start_time = None
        self.last_logged_step = -1
        self.log_path = log_path
        if os.path.exists(log_path):
            os.remove(log_path)
    def _progress_bar(self, current, total, width=40):
        filled = int(width * current / max(1, total))
        return '█' * filled + '░' * (width - filled)
    def _log(self, msg):
        with open(self.log_path, 'a', encoding='utf-8') as f:
            f.write(msg + '\n')
    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
        self._log('TRAINING STARTED')
    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        current_step = state.global_step
        if current_step == self.last_logged_step:
            return
        self.last_logged_step = current_step
        elapsed = time.time() - self.start_time
        total = self.total_steps if self.total_steps else max(1, current_step)
        progress_pct = (current_step / total) * 100
        if current_step > 0:
            avg_time_per_step = elapsed / current_step
            remaining_steps = max(0, total - current_step)
            eta_minutes = (avg_time_per_step * remaining_steps) / 60
        else:
            eta_minutes = 0
        bar = self._progress_bar(current_step, total)
        loss = logs.get('loss')
        msg = f'Step {current_step}/{total} ({progress_pct:.1f}%) | Loss: {loss:.4f} | Elapsed: {elapsed/60:.1f} min | ETA: {eta_minutes:.1f} min'
        print('\n' + '='*80)
        print(f'📊 {msg}')
        print(f'[{bar}]')
        print('='*80)
        self._log(msg)
    def on_save(self, args, state, control, **kwargs):
        elapsed = time.time() - self.start_time if self.start_time else 0
        ckpt = f'checkpoint-{state.global_step}'
        msg = f'Checkpoint saved: {ckpt} | Elapsed: {elapsed/60:.1f} min'
        print(msg)
        self._log(msg)
    def on_train_end(self, args, state, control, **kwargs):
        total_time = time.time() - self.start_time if self.start_time else 0
        msg = f'TRAINING COMPLETE! Total Time: {total_time/60:.1f} min'
        print(msg)
        self._log(msg)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=2048, padding='max_length')

print('Tokenizing dataset...')
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
checkpoint_dir = '/content/n8n-workflow-generator'
resume_from_checkpoint = None
if os.path.exists(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if checkpoints:
        latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))[-1]
        resume_from_checkpoint = os.path.join(checkpoint_dir, latest_checkpoint)
        print(f'Found checkpoint: {latest_checkpoint}, resuming...')
total_steps = len(train_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[TextProgressCallback(total_steps, training_args.num_train_epochs, '/content/training_progress.log')],
)
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

Tokenizing dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
1,1.590800
5,1.597000
10,1.635500
15,1.429500
20,1.393800
25,1.266300
30,1.264100
35,1.249600
40,1.113700
45,1.113200



📊 Step 1/375 (0.3%) | Loss: 1.5908 | Elapsed: 1.1 min | ETA: 422.0 min
[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 5/375 (1.3%) | Loss: 1.5970 | Elapsed: 5.9 min | ETA: 436.7 min
[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 10/375 (2.7%) | Loss: 1.6355 | Elapsed: 12.2 min | ETA: 445.3 min
[█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 15/375 (4.0%) | Loss: 1.4295 | Elapsed: 18.8 min | ETA: 451.8 min
[█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 20/375 (5.3%) | Loss: 1.3938 | Elapsed: 25.2 min | ETA: 446.7 min
[██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 25/375 (6.7%) | Loss: 1.2663 | Elapsed: 31.3 min | ETA: 438.5 min
[██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]
Checkpoint saved: checkpoint-25 | Elapsed: 31.3 min

📊 Step 30/375 (8.0%) | Loss: 1.2641 | Elapsed: 37.5 min | ETA: 431.5 min
[███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 35/375 (9.3%) | Loss: 1.2496 | Elapsed: 43.4 min | ETA: 422.0 min
[███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]

📊 Step 40/375 (

TrainOutput(global_step=375, training_loss=0.9577280069986979, metrics={'train_runtime': 28175.2544, 'train_samples_per_second': 0.106, 'train_steps_per_second': 0.013, 'total_flos': 2.62629884952576e+17, 'train_loss': 0.9577280069986979, 'epoch': 3.0})

In [10]:
# Save final model
output_dir = '/kaggle/working/n8n-workflow-generator-final'
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f'Model saved to {output_dir}')

Model saved to /kaggle/working/n8n-workflow-generator-final
